In [1]:
import sys
from ftplib import FTP #import the ftp library
import re 
import os
import pandas as pd
import numpy as np

In [2]:
#cwd = os.getcwd()
#print cwd

In [3]:
## different classes
#t='Dinophyceae'
#t='Bacillariophyta'
t='Haptophyta'
#t='Raphidophyceae'
need='mmetsp_taxonomy.txt'

### MAFFT and HMMbuild in shell

In [4]:
###run mafft and hmmbuild first to make script faster
#os.system('./mafft_hmmbuil.sh')

### Species list for HMMFUNCTION

will not retrive files, but will create a list with names that we can pass to bash and other functions

In [4]:
mt=open('mmetsp_taxonomy.txt','r')
g=[] #make an empty list to store genus names
for line in mt:
    if re.search(t,line): #if taxa name in line
        g= g+line.split('\t')[7:8]#pull out the 8th field should be genus, keeping as list

g=set(g) #keep only unique genus names
print g
#close the taxonomy file
mt.close()

set(['Exanthemachrysis', 'Gephyrocapsa', 'Chrysochromulina', 'Isochrysis', 'Pleurochrysis', 'Pavlova', 'Phaeocystis', 'Coccolithus', 'Imantonia', 'Unidentified eukaryote', 'Prymnesium', 'Chrysoculter', 'Scyphosphaera', 'Emiliania', 'Calcidiscus'])


In [5]:
ftp= FTP('ftp.imicrobe.us') #set home ftp server
ftp.login() #log in
ftp.cwd('camera/combined_assemblies') #ch

files=ftp.nlst() #make a list of all files and directories in wd
delimiter=' '
all=delimiter.join(files)

names=[]

for genus in g:
    string= genus+"\S*.pep.fa.gz"
    taxafiles=re.findall(string, all)
    #print "{} files matching genus=".format(len(taxafiles))+genus
    #print taxafiles
    if len(taxafiles) > 0:
        for filex in taxafiles:
            command = "RETR "+filex
            outfile = filex
            #ftp.retrbinary(command, open(outfile, 'wb').write)
            names.append(outfile)
            
ftp.quit()

print names


['Gephyrocapsa-oceanica-RCC1303.pep.fa.gz', 'Chrysochromulina-polylepis-CCMP1757.pep.fa.gz', 'Isochrysis-galbana-CCMP1323.pep.fa.gz', 'Isochrysis-sp-CCMP1244.pep.fa.gz', 'Isochrysis-sp-CCMP1324.pep.fa.gz', 'Pleurochrysis-carterae-CCMP645.pep.fa.gz', 'Pavlova-sp-CCMP459.pep.fa.gz', 'Prymnesium-parvum-Texoma1.pep.fa.gz', 'Emiliania-huxleyi-374.pep.fa.gz', 'Emiliania-huxleyi-379.pep.fa.gz', 'Emiliania-huxleyi-CCMP370.pep.fa.gz', 'Emiliania-huxleyi-PLYM219.pep.fa.gz']


In [8]:
names=names[:2]
print names

['Gephyrocapsa-oceanica-RCC1303.pep.fa.gz', 'Chrysochromulina-polylepis-CCMP1757.pep.fa.gz']


### Retrieve PEP.fa in shell

In [147]:
###run retriver in shell
os.system('python ./MMETSP_sample_import.py {} {}'.format(t,need))

0

#### Fix error on Dinos

In [6]:
##the last species is not available for some reason so I removed it
print len(names)

if t=='Dinophyceae':
    names.remove('Durinskia-baltica-CSIRO_CS-38.pep.fa.gz')
    names.remove('Oxyrrhis-marina-CCMP1795.pep.fa.gz')
    names.remove('Alexandrium-fundyense-CCMP1719.pep.fa.gz')
    
print len(names)

30
27


### GET counts

In [115]:
names=[i.split('.')[0] for i in names] #removes .pep.fa.gz from the names

ftp= FTP('ftp.imicrobe.us') #set ftp server
ftp.login() #log in
ftp.cwd('camera/combined_assemblies') #change directory

t='/Users/maria_hernandez/Documents/Big_Data3050/CMM_MoreSP/'  #location for files
for ID in names:
    #change to taxa directory/readcounts
    ripdir= ID+"/readcounts"
    ftp.cwd(ripdir) #change directory
    savefile= t+ID+"_cds_counts.txt" #saves files with unique names
    ftp.retrbinary('RETR cds.dat', open(savefile, 'wb').write)
    ftp.cwd("~/camera/combined_assemblies") #change directory to restart loop in right place
    
ftp.quit()   

In [16]:
names='Chrysochromulina-polylepis-CCMP1757.pep.fa.gz'

### RUN HMM in shell

In [9]:
for i in names:
    os.system('./MAGIC_HMM.sh {}'.format(i))

In [22]:
for i in names:
    os.system('./MAGIC_one.sh {}'.format(i))

### HMM READ

In [8]:
def readHMM(Organism,gene_name):
    """ Takes in organism and gene_name from HMM results and makes a table.
    HMM results from --tblout that have the following name organism_genename_HMM.csv
    Note: pep.fa files differ in structure and it can affect how the HMM output is written. If you can't read the file in 
    modify the fuction"""

    hold=pd.read_csv('{}_{}_HMM.csv'.format(Organism,gene_name),delim_whitespace=True,engine='python',skiprows=2,skipfooter=10)
    #empty files have 12 rows so the following if statement will only work on files that are not empty
    
    if hold.shape[0]!=0:
        readX=pd.read_csv('{}_{}_HMM.csv'.format(Organism,gene_name),delim_whitespace=True,engine='python',skiprows=3,
                          header=None ,skipfooter=10)
        
        samplenames=[]
        for i in np.arange(1,readX.shape[1]+1):
            samplenames.append(str(i))

        readX.columns=samplenames
        
        new=pd.DataFrame()
        
        new['CAMPEPid']=readX['1']
        new['contig']=readX['19']
        new['Evalue']=readX['5']
        new['Annotation']='{}'.format(gene_name)

        new.contig=new.contig.str.split("|").str[1]
        new.contig=new.contig.str.split("_").str[0]
        return new
        #return readX

In [9]:
readHMM('Chrysochromulina-polylepis-CCMP1757.pep.fa.gz','Bestrophin').head()

,CAMPEPid,contig,Evalue,Annotation
0,CAMPEP_0193794506,169964,1.600000e-125,Bestrophin
1,CAMPEP_0193783668,140936,1.900000e-44,Bestrophin
2,CAMPEP_0193795430,170428,1.400000e-43,Bestrophin
3,CAMPEP_0193788850,155911,1.300000e-25,Bestrophin
4,CAMPEP_0193725542,15531,4.000000e-24,Bestrophin


In [30]:
def makeHMM(Organism,gene_type):
    """Combines the plots from each gene_type into a table"""
    if gene_type=='CF':
        a=readHMM(Organism,'PK')
        b=readHMM(Organism,'PEPC')
        c=readHMM(Organism,'PEPCK')
        d=readHMM(Organism,'PK')
        e=readHMM(Organism,'MDH')
        f=readHMM(Organism,'OMT')
        g=readHMM(Organism,'PYC')
        h=readHMM(Organism,'PPDK')
        i=readHMM(Organism,'ME')
        
        frames = [a,b,c,d,e,f,g,i,h]
        result = pd.concat(frames)
        return result
    if gene_type=='PR':
        a=readHMM(Organism,'SHMT')
        b=readHMM(Organism,'GOX')
        c=readHMM(Organism,'GDCT')
        d=readHMM(Organism,'PGP')
        e=readHMM(Organism,'ICL')
        f=readHMM(Organism,'GCL')
        g=readHMM(Organism,'HR')
        h=readHMM(Organism,'SPT')
        i=readHMM(Organism,'TSR')
        j=readHMM(Organism,'MS')
        k=readHMM(Organism,'GlcDH')
        l=readHMM(Organism,'ALAT_GGAT')
        m=readHMM(Organism,'GK')
        
        frames = [a,b,c,d,e,f,g,h,i,j,k,l,m]
        result = pd.concat(frames)
        return result
    if gene_type=='BP':
        a=readHMM(Organism,'CA_alpha')
        b=readHMM(Organism,'CA_delta')
        c=readHMM(Organism,'CA_beta')
        #d=readHMM(Organism,'Ca_zeta')
        e=readHMM(Organism,'Bestrophin')
        f=readHMM(Organism,'SLC4')

        frames = [a,b,c,e,f]
        result = pd.concat(frames)
        return result

In [13]:
makeHMM('Chrysochromulina-polylepis-CCMP1757.pep.fa.gz','BP')

,CAMPEPid,contig,Evalue,Annotation
0,CAMPEP_0193716998,10130_1,8.300000e-164,CA_delta
1,CAMPEP_0193720114,12139_1,2.500000e-144,CA_delta
2,CAMPEP_0193740950,25073_1,2.000000e-112,CA_delta
3,CAMPEP_0193744210,27355_1,1.700000e-97,CA_delta
0,CAMPEP_0193731632,19523_1,7.400000e-17,CA_beta
1,CAMPEP_0193739048,23878_1,4.500000e-07,CA_beta
0,CAMPEP_0193794506,169964_1,1.600000e-125,Bestrophin
1,CAMPEP_0193783668,140936_1,1.900000e-44,Bestrophin
2,CAMPEP_0193795430,170428_1,1.400000e-43,Bestrophin
3,CAMPEP_0193788850,155911_1,1.300000e-25,Bestrophin


In [31]:
def HMMcmp(Organism,gene_type):
    result=makeHMM(Organism,gene_type)
    contig_EV=dict(zip(result.contig,result.Evalue))
    contig_Annot=dict(zip(result.contig,result.Annotation))
    
    
    Organism = re.sub('.pep.fa.gz', '', Organism)
    
    expression=pd.read_csv('{}_cds_counts.txt'.format(Organism),delimiter='\t',index_col=0)
 
    expression['log2CPM']=np.log2(expression.sum(axis=1)/1000000)
    expression.index= expression.index.str.split("|").str[1]
    contig_CMP=dict(zip(expression.index,expression.log2CPM))
    
    Contig=[]
    Evalue=[]
    Annotation=[]
    log2CPM=[]
    
    for i in expression.index:
        a=contig_CMP.get(i)
        if a>-16 or a=="-inf" or a=="inf":
            Contig.append(i)
            Evalue.append(contig_EV.get(i))
            Annotation.append(contig_Annot.get(i))
            log2CPM.append(a)
              

    out=pd.DataFrame()
    out['Contig']=Contig
    out['Evalue']=Evalue
    out['Annotation']=Annotation
    out['log2CPM']=log2CPM
        
    return out

In [15]:
HMMcmp('Chrysochromulina-polylepis-CCMP1757.pep.fa.gz','BP')

,Contig,Evalue,Annotation,log2CPM
0,3_1,NaN,None,-6.713308
1,1_1,NaN,None,-6.339929
2,4_1,NaN,None,-9.924541
3,6_1,NaN,None,-14.844106
4,5_1,NaN,None,-9.441721
5,8_1,NaN,None,-10.127438
6,7_1,NaN,None,-11.431723
7,2_1,NaN,None,-7.236558
8,9_1,NaN,None,-15.609640
9,10_1,NaN,None,-15.231129


In [32]:
def HMMclean(Organism):
    '''Will create an HMM table and filter out values that do not meet an e-value'''
    a=HMMcmp(Organism,'BP')
    b=HMMcmp(Organism,'CF')
    c=HMMcmp(Organism,'PR')
    
    frames=[a,b,c]
    HMM= pd.concat(frames)
    
    #print HMM
    
    HMM=HMM[HMM['Evalue'] < .00001]
    
    #return HMM.drop_duplicates()
    
    Genes= np.unique(HMM['Annotation'])
    A=pd.DataFrame(0,index=[Organism],columns=Genes)
    
    for j in Genes:
        counts=HMM.Annotation.value_counts()[j]
        A[j]= counts
           
    return A
        

In [17]:
HMMclean('Chrysochromulina-polylepis-CCMP1757.pep.fa.gz')

,ALAT_GGAT,Bestrophin,CA_beta,CA_delta,GCL,GDCT,GlcDH,HR,ICL,MDH,...,OMT,PEPC,PEPCK,PGP,PK,PYC,SHMT,SLC4,SPT,TSR
Chrysochromulina-polylepis-CCMP1757.pep.fa.gz,8,48,2,4,2,1,4,9,3,2,...,47,1,1,11,9,10,4,5,3,7


In [18]:
print names

['Gephyrocapsa-oceanica-RCC1303.pep.fa.gz', 'Chrysochromulina-polylepis-CCMP1757.pep.fa.gz']


In [33]:
def gen_type(gene_type):
    """ Allows you to extract specific gene_types"""
    
    if gene_type=='BP':
            fixed=['SLC4','Bestrophin','CA_alpha','CA_beta','CA_delta','CA_zeta']
    if gene_type=='PR':
            fixed=['PGP','GOX','SPT','ALAT_GGAT','GDCT','SHMT','HR','GK','GlcDH','MS','ICL','GCL','TSR']
    if gene_type=='CF':
            fixed=['PK','PEPC','PEPCK','MDH','OMT','ME','PPDK','PYC']
            
    A=pd.DataFrame(0, index=names, columns=fixed)
    
    for i,j in enumerate(names):
        for k in fixed:
            B=HMMclean(j)
            #print B
            if k in list(B):
                A[k][i]=B[k]
                
    A.index = A.index.str.split('.').str[0]
    
    C=A.transpose()
    #C.to_csv('{}_{}_GeneCountHMM.csv'.format(t,gene_type))
    
    return A.transpose()

In [30]:
#gen_type('BP')

### All sections in one table

In [34]:
a=gen_type('BP')
b=gen_type('CF')
c=gen_type('PR')

framesX=(a,b,c)
outFrame=pd.concat(framesX)

outFrame.to_csv('COP_ADD.csv')

print outFrame

            Gephyrocapsa-oceanica-RCC1303  \
SLC4                                    6   
Bestrophin                             37   
CA_alpha                                2   
CA_beta                                 0   
CA_delta                                2   
CA_zeta                                 0   
PK                                      4   
PEPC                                    1   
PEPCK                                   1   
MDH                                     4   
OMT                                    69   
ME                                      2   
PPDK                                    0   
PYC                                    16   
PGP                                     9   
GOX                                     1   
SPT                                     2   
ALAT_GGAT                              13   
GDCT                                    4   
SHMT                                    4   
HR                                     10   
GK        

In [24]:
print outFrame['Chrysochromulina-polylepis-CCMP1757']

SLC4           5
Bestrophin    48
CA_alpha       0
CA_beta        2
CA_delta       4
CA_zeta        0
PK             9
PEPC           1
PEPCK          1
MDH            2
OMT           47
ME             3
PPDK           0
PYC           10
PGP           11
GOX            0
SPT            3
ALAT_GGAT      8
GDCT           1
SHMT           4
HR             9
GK             0
GlcDH          4
MS             0
ICL            3
GCL            2
TSR            7
Name: Chrysochromulina-polylepis-CCMP1757, dtype: int64
